# String Processing, Listen und Schleifen
__Anwendungsfall:__ Sie bekommen eine Textdatei mit Kontakten (Name, Email-Adresse, Geschlecht, ...) und möchten eine personalisierte Email an jeden Kontakt senden.

## Aufgabe
Bis auf 3 Code-Stellen in der Funktion __erzeuge_adressbuch_aus()__, an denen Sie jeweils einen String splitten müssen, ist das Email-Sende-Notebook schon fertig.

Diese Code-Stellen sind mit __#TO DO:__ gekennzeichnet. Ob sie "richtig splitten" und damit die Aufgabe erfüllt haben, können sie erkennen, wenn sie die beiden zugehörigen Tests laufen lassen.

PS: Dieses Notebook würde Emails senden, wenn sie korrekte Werte bei den "Zugangsdaten" angeben und es lokal von ihrem Computer zu Hause oder bspw. unter https://colab.research.google.com/ ausführen. Seien sie aber vorsichtig mit ihrem Email-Passwort, wenn sie Cloud-Dienste zur Ausführung des Notebooks nutzen!

## Programm
1. Ein mehrzeiliger String (KONTAKTE) wird in eine Liste von Listen (hier adressbuch genannt) umgewandelt
2. Das Adressbuch wird benutzt, um für jeden Empfänger eine Email zu erstellen. Emails sind Strings. Diese Emails werden wieder in einer Liste gespeichert (hier briefkorb genannt)
3. Im letzen Schritt werden alle Emails an einen SMTP-Server übergeben - genau, wie es andere Email-Programme auch machen (Simple Mail Transport Protocoll)

In [ ]:
# Dieser Code läuft natürlich erst, wenn die Zellen weiter unten ausgeführt wurden.
# Die Funktionen erzeuge_adressbuch_aus(), erstelle_emails_an() und briefe_einwerfen()
# müssen bekannt sein.

adressbuch = erzeuge_adressbuch_aus(KONTAKTE)
print("{} Adressen verarbeitet".format(len(adressbuch)))

briefkorb = erstelle_emails_an(adressbuch)
print("{} Emails erstellt".format(len(briefkorb)))

protokoll = briefe_einwerfen(briefkorb)
print("{} Emails versendet".format(len(protokoll)))

In [ ]:
for entry in protokoll:
  print("To: {}\tStatus: {}".format(entry[0]['To'],entry[1]))

### Konfiguration

#### Unter welchem Namen wollen wir versenden:

In [ ]:
ABSENDER = "Rocco Melzian <rocco.melzian@kantiolten.ch>" # was auf dem Couvert steht

#### Welchen Inhalt die Email haben soll:

In [ ]:
BETREFF = "Vorsicht! Geheime Verschlusssache"

NACHRICHT = """
{ANREDE} {VORNAME}

Die geheimen Koordinaten lauten ...

{GRUSSFORMEL} {UNTERZEICHNER}
"""

#### An wen soll die Email gehen:
Diese Daten werden später aus einer Datei direkt eingelesen werden - dann können es beliebig viele Zeilen sein.

In [ ]:
KONTAKTE = """
Rocco Melzian;m;rocco.melzian@kantiolten.ch
Rocco Melzian;w;rocco@melzian.ch
Rocco Melzian;d;zero-overhead@melzian.eu
Rocco Melzian;;zero-overhead@melzian.eu
"""
ANZAHL_SPALTEN_KONTAKTE = 3 #1. Name Vorname; 2. Geschlecht; 3. Email

#### Unsere Zugangsdaten zu einem "Briefkasten":
Die Zeiten, wo man in jeden "Briefkasten" ohne Authentifizierung "Post" für jeden Absender und jeden Empfänger einwerfen konnte, sind längst vorbei.

https://de.wikipedia.org/wiki/Simple_Mail_Transfer_Protocol

In [ ]:
from getpass import getpass

MY_SMTP_SERVER = "smtp.office365.com"        # Postkasten
MY_SMTP_PORT = 587                           # sozusagen der Postkastenschlitz
MY_SMTP_USER = "rocco.melzian@kantiolten.ch" # Zugangsdaten Postkasten

try: MY_SMTP_PASSWORD
except NameError: MY_SMTP_PASSWORD = getpass("Passwort für {}: ".format(MY_SMTP_USER)) # wenn input() statt getpass() verwendet wird, wird das Passwort während der Eingabe angezeigt

### Implementierung

#### erzeuge_adressbuch_aus()

In [ ]:
def erzeuge_adressbuch_aus(kontakte_string):
    
    #TO DO: Splitten sie den Inhalt von kontakte_string am Zeilenumbruch
    kontakte_liste = kontakte_string.split(???) 
    
    adressbuch = []
    for zeile in kontakte_liste:
        zeile = zeile.rstrip()                  # Zeilenumbruch '\n' entfernen

        #TO DO: Splitten sie den Inhalt von daten am ;
        daten = zeile???

        if(len(daten) != ANZAHL_SPALTEN_KONTAKTE):  # Nur vollständige Kontakte
            continue

        name_string = daten[0]
        geschlecht = daten[1]
        email = daten[2]

        #TO DO: Splitten sie name_string am Leerzeichen (default);
        vorname, name = name_string???
        
        eintrag = [name, vorname, geschlecht, email]
        
        adressbuch.append(eintrag)

    return(adressbuch)

#### Tests für die Funktion "erzeuge_adressbuch_aus()"

In [ ]:
test_message = "Test {} : {}"

In [ ]:
testnummer = 1
testdaten1_kontakte = "Rocco Melzian;m;rocco.melzian@kantiolten.ch"
testdaten1_erwartet = [
    ['Melzian', 'Rocco', 'm', 'rocco.melzian@kantiolten.ch']
]
testdaten1_erzeugt = erzeuge_adressbuch_aus(testdaten1_kontakte)
ergebnis = testdaten1_erwartet == testdaten1_erzeugt
print(test_message.format(testnummer, ergebnis))

In [ ]:
testnummer = 2
testdaten2_kontakte = """
Rocco Melzian;m;rocco.melzian@kantiolten.ch
Rocco Melzian;w;rocco@melzian.ch
"""
testdaten2_erwartet = [
    ['Melzian', 'Rocco', 'm', 'rocco.melzian@kantiolten.ch'],
    ['Melzian', 'Rocco', 'w', 'rocco@melzian.ch']
]
testdaten2_erzeugt = erzeuge_adressbuch_aus(testdaten2_kontakte)
ergebnis = testdaten2_erwartet == testdaten2_erzeugt
print(test_message.format(testnummer, ergebnis))

#### erstelle_emails_an()
und Helfer-Funktionen. Die Funktion text_fuer() müsste verbessert werden, wenn bspw. die Anrede abhängig vom Geschlecht sein soll.

In [ ]:
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def erstelle_emails_an(empfaengerinnen):
    sammelbox = []
    for empfaenger in empfaengerinnen:
        individualierter_text = text_fuer(empfaenger)
        diese_email = beschriften_fuer(empfaenger)
        diese_email.attach(MIMEText(individualierter_text, 'plain'))
        sammelbox.append(diese_email)
    return(sammelbox)

def text_fuer(empfaengerin):
    
        name,vorname,geschlecht,email = empfaengerin

        text = NACHRICHT.format(# Platzhalter in NACHRICHT ersetzen
                            ANREDE="Liebe",
                            VORNAME=vorname,
                            GRUSSFORMEL="Hastalavista",
                            UNTERZEICHNER="Ich")
        return(text)

def beschriften_fuer(empfaenger):
    
    name = empfaenger[0]
    vorname = empfaenger[1]
    email = empfaenger[3]
    empfaengerin_formatiert = "{} {} <{}>".format(vorname, name, email)

    msg = MIMEMultipart() # Email-Grundgerüst
    
    # Was wir auf die Postkarte bei "Empfänger" schreiben können:
    # https://de.wikipedia.org/wiki/Header_(E-Mail)
    msg['From'] = ABSENDER
    msg['To'] = empfaengerin_formatiert
    msg['Subject'] = BETREFF
    return(msg)

#### briefe_einwerfen()
Simple Mail Transfer Protocol - "Briefe in den Postkasten einwerfen"

In [ ]:
import smtplib

def briefe_einwerfen(briefe):
    conn = smtplib.SMTP(MY_SMTP_SERVER,MY_SMTP_PORT)
    conn.starttls()
    
    try: 
        conn.login(MY_SMTP_USER,MY_SMTP_PASSWORD)
    except: 
        del globals()['MY_SMTP_PASSWORD']  #reset password
        print("Server/Username/Passwort wrong? Passwort has been reset - enter it again.")
        return ""
    
    log = []
    for brief in briefe:
        status = conn.send_message(brief)
        entry = [brief, status]
        log.append(entry)
    conn.close()
    return(log)

## Verwendete Konzepte:
- Definieren von Variablen und Konstanten
- Definieren von mehrzeiligen String-Variablen
- Zeilenumbrüche \n
- User input()
- String.format() mit Platzhaltern nutzen
- Importieren von Bibliotheken
- Importierte Funktionen nutzen
- Funktionen mit Parametern selbst schreiben und anwenden
- String.split() nutzen
- String.format() mit benannten Platzhaltern nutzen
- For-Schleifen nutzen
- Listen erstellen
- Listen sukzessive befüllen
- Länge von Listen bestimmen
- Elemente aus Listen auslesen